# TOI-5807.01 End-to-End Tutorial (TIC 188646744)

This is the consolidated, end-to-end TOI-5807.01 workflow and is intended to eventually replace:

- `04-real-candidate-validation.ipynb`
- `05-extended-metrics.ipynb`
- `06-toi-5807-robustness.ipynb`
- `07-toi-5807-consolidated-validation.ipynb`

It uses the newer researcher UX APIs to minimize notebook glue:

- dataset loading: `btv.load_tutorial_target(...)`
- one-call orchestration: `btv.run_candidate_workflow(...)`
- per-sector reruns: `btv.per_sector_vet(...)`
- reporting + exports: `btv.format_vetting_table(...)`, `btv.export_bundle(...)`
- FPP convenience: `btv.hydrate_cache_from_dataset(...)`, `btv.load_contrast_curve_exofop_tbl(...)`

This notebook is **metrics-first**: it returns quantitative diagnostics and provenance. It does not add new decision thresholds.

Traceability goals:

- All key results are printed as machine-readable dicts (easy to compare)
- FPP runs can be made reproducible via `FPP_SEED` + `FPP_REPLICATES`
- Optional network-dependent steps are clearly gated


In [ ]:
from __future__ import annotations

import tempfile
from pathlib import Path

import numpy as np

import tess_vetter.api as btv

# Optional network helpers
from tess_vetter.api.catalogs import fetch_exofop_toi_table
from tess_vetter.api.fpp import calculate_fpp

# ----------------------------------------------------------------------------
# Controls
# ----------------------------------------------------------------------------
NETWORK = True  # enables V06/V07 and FPP upstream calls (Gaia/TRILEGAL)
RUN_TRANSIT_FIT = False  # optional; requires extra deps (e.g. batman)
RUN_FPP = False  # optional; can take minutes and has MC variance
RUN_MULTI_SECTOR_TPFS = False  # requires lightkurve + network

# FPP reproducibility controls (when RUN_FPP=True)
FPP_PRESET = "standard"  # "fast" or "standard"
FPP_REPLICATES = 1
FPP_SEED = 42
FPP_TIMEOUT_SECONDS = 1800

TIC_ID = 188646744
TOI_LABEL = "TOI-5807.01"

# Coordinates for catalog queries when NETWORK=True
RA_DEG = 304.12005
DEC_DEG = 11.08344


## Run configuration (for traceability)

This cell prints the key toggles and versions so an astrophysicist can reproduce the same pathway.


In [ ]:
import importlib.metadata

def _pkg_version(name: str) -> str | None:
    try:
        return importlib.metadata.version(name)
    except Exception:
        return None

print(
    {
        "NETWORK": NETWORK,
        "RUN_TRANSIT_FIT": RUN_TRANSIT_FIT,
        "RUN_MULTI_SECTOR_TPFS": RUN_MULTI_SECTOR_TPFS,
        "RUN_FPP": RUN_FPP,
        "FPP_PRESET": FPP_PRESET,
        "FPP_REPLICATES": FPP_REPLICATES,
        "FPP_SEED": FPP_SEED,
        "FPP_TIMEOUT_SECONDS": FPP_TIMEOUT_SECONDS,
        "versions": {
            "tess-vetter": _pkg_version("tess-vetter"),
            "numpy": _pkg_version("numpy"),
            "lightkurve": _pkg_version("lightkurve"),
            "wotan": _pkg_version("wotan"),
            "astropy": _pkg_version("astropy"),
        },
    }
)


## 1) Load the bundled tutorial dataset

This repo includes a tutorial dataset folder under `docs/tutorials/data/tic188646744/`.

It contains:

- per-sector PDCSAP light curves (`sector*_pdcsap.csv`)
- a representative TPF stamp (`sector83_tpf.npz`)
- a PHARO AO contrast curve (`PHARO_Kcont_plot.tbl`)

<details>
<summary><b>Expected Output</b> (click to expand)</summary>

You should see a dataset summary similar to:

```
{
  'schema_version': 1,
  'root': '.../docs/tutorials/data/tic188646744',
  'sectors_lc': [55, 75, 82, 83],
  'sectors_tpf': [83],
  'artifacts': ['files']
}
```
</details>


In [ ]:
ds = btv.load_tutorial_target("tic188646744")
ds.summary()

## 2) (Optional) Pull live ExoFOP values

If `NETWORK=True`, we can query the ExoFOP TOI table to display current ephemeris/stellar context.

For reproducibility, the rest of this notebook uses offline defaults if the query is skipped.


In [ ]:
toi = None
if NETWORK:
    toi_table = fetch_exofop_toi_table()
    matches = toi_table.entries_for_tic(TIC_ID)
    toi = matches[0] if matches else None
    print(f"ExoFOP entries for TIC {TIC_ID}: {len(matches)}")
    if toi:
        {k: toi.get(k) for k in ["toi", "period_days", "epoch_bjd", "duration_hours", "depth_ppm", "tess_disposition", "tfopwg_disposition"]}
else:
    print("Skipping ExoFOP (set NETWORK=True).")

## 3) Candidate ephemeris + stellar parameters

We use ExoFOP values when available; otherwise a stable offline fallback.

<details>
<summary><b>Expected Output</b> (click to expand)</summary>

If running offline (or if ExoFOP changes), you should see something close to:

```
{
  'period_days': 14.2423724,
  't0_btjd': 3540.26317,
  'duration_hours': 4.046,
  'depth_ppm': 253.0,
  'stellar_radius_rsun': 1.65,
  'stellar_mass_msun': 1.47
}
```
</details>


In [ ]:
def _safe_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


# Offline fallback values (BTJD = BJD - 2457000)
PERIOD_DAYS = _safe_float(toi.get("period_days"), 14.2423724) if toi else 14.2423724
epoch_bjd = _safe_float(toi.get("epoch_bjd"), 2460540.26317) if toi else 2460540.26317
T0_BTJD = (epoch_bjd - 2457000.0) if epoch_bjd and epoch_bjd > 2_450_000 else (epoch_bjd or 3540.26317)
DURATION_HOURS = _safe_float(toi.get("duration_hours"), 4.046) if toi else 4.046
DEPTH_PPM = _safe_float(toi.get("depth_ppm"), 253.0) if toi else 253.0

stellar = btv.StellarParams(
    radius=_safe_float(toi.get("stellar_radius_r_sun"), 1.65) if toi else 1.65,
    mass=_safe_float(toi.get("stellar_mass_m_sun"), 1.47) if toi else 1.47,
    teff=_safe_float(toi.get("stellar_eff_temp_k"), 6816.0) if toi else 6816.0,
    logg=_safe_float(toi.get("stellar_logg"), 4.17) if toi else 4.17,
)

ephem0 = btv.Ephemeris(period_days=PERIOD_DAYS, t0_btjd=T0_BTJD, duration_hours=DURATION_HOURS)
cand0 = btv.Candidate(ephemeris=ephem0, depth_ppm=DEPTH_PPM)

{
    "period_days": ephem0.period_days,
    "t0_btjd": ephem0.t0_btjd,
    "duration_hours": ephem0.duration_hours,
    "depth_ppm": cand0.depth_ppm,
    "stellar_radius_rsun": stellar.radius,
    "stellar_mass_msun": stellar.mass,
}

## 4) (Optional) Refine ephemeris via transit fit

`btv.fit_transit(...)` uses optional dependencies (e.g. `batman`).

This notebook defaults to **not** fitting (for reproducibility). To enable fitting, set:

- `RUN_TRANSIT_FIT = True`


In [ ]:
stitched = btv.stitch_lightcurves(
    [
        {
            "time": np.asarray(lc.time, dtype=np.float64),
            "flux": np.asarray(lc.flux, dtype=np.float64),
            "flux_err": np.asarray(lc.flux_err, dtype=np.float64),
            "sector": int(sector),
            "quality": np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(ds.lc_by_sector.items())
    ]
)

lc_stitched = btv.LightCurve(
    time=stitched.time,
    flux=stitched.flux,
    flux_err=stitched.flux_err,
    quality=stitched.quality,
)

if RUN_TRANSIT_FIT:
    fit = btv.fit_transit(lc_stitched, cand0, stellar)
    if getattr(fit, "status", "success") != "success" or float(getattr(fit, "duration_hours", 0.0)) <= 0:
        ephem = ephem0
        candidate = cand0
    else:
        ephem = btv.Ephemeris(
            period_days=PERIOD_DAYS,
            t0_btjd=T0_BTJD + float(fit.t0_offset),
            duration_hours=float(fit.duration_hours),
        )
        candidate = btv.Candidate(ephemeris=ephem, depth_ppm=float(fit.transit_depth_ppm))
else:
    fit = None
    ephem = ephem0
    candidate = cand0

{
    "fit_status": None if fit is None else getattr(fit, "status", None),
    "fit_error": None if fit is None else getattr(fit, "error_message", None),
    "period_days": ephem.period_days,
    "t0_btjd": ephem.t0_btjd,
    "duration_hours": ephem.duration_hours,
    "depth_ppm": candidate.depth_ppm,
}

## 5) Baseline vetting + per-sector rerun (`preset="default"`)

This runs the default preset on a stitched light curve and also runs per-sector bundles.

Notes:

- This tutorial runs LC-only + catalog + exovetter checks in the stitched/global bundle.
- Pixel checks (V08–V10) are demonstrated later as a per-sector workflow once you have matching per-sector TPFs.

<details>
<summary><b>Expected Output</b> (click to expand)</summary>

You should see a full vetting table printout plus a compact summary dict.

Representative (from prior TOI-5807.01 runs):

```
V01 delta_sigma ~ 1.2
V02 secondary_depth_sigma ~ 1.1
```

Exact values can vary if you change the ephemeris, enable fitting, or change sectors.
</details>


In [ ]:
wf_default = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="default",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(wf_default.bundle))

r01 = wf_default.bundle.get_result("V01")
r02 = wf_default.bundle.get_result("V02")

{
    "counts": {
        "checks": len(wf_default.bundle.results),
        "ok": wf_default.bundle.n_passed,
        "error": wf_default.bundle.n_failed,
        "skipped": wf_default.bundle.n_unknown,
    },
    "key_metrics": {
        "V01_delta_sigma": None if r01 is None else r01.metrics.get("delta_sigma"),
        "V02_secondary_depth_sigma": None if r02 is None else r02.metrics.get("secondary_depth_sigma"),
    },
    "per_sector_summary": wf_default.per_sector.summary_records if wf_default.per_sector else None,
}

## 6) Extended metrics (`preset="extended"`, V16–V21)

Extended checks add metrics-only diagnostics and may skip based on available inputs.

Key things to look for in TOI-5807.01:

- **V16** model competition stability (especially after detrending)
- **V19** phase-shift event count / max significance (prompts inspection)
- **Per-sector ephemeris metrics**: depth_hat_ppm and depth_sigma_ppm by sector


In [ ]:
wf_ext = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="extended",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(wf_ext.bundle))

ext_summary = btv.summarize_bundle(
    wf_ext.bundle,
    check_ids=["V16", "V17", "V18", "V19", "V20", "V21"],
    include_metrics=True,
    include_flags=True,
    include_notes=False,
)

r16 = wf_ext.bundle.get_result("V16")
r19 = wf_ext.bundle.get_result("V19")

{
    "extended_summary": ext_summary,
    "V16": None if r16 is None else {
        "winner": r16.metrics.get("winner"),
        "label": r16.metrics.get("model_competition_label"),
        "winner_margin_bic": r16.metrics.get("winner_margin_bic"),
        "artifact_risk": r16.metrics.get("artifact_risk"),
    },
    "V19": None if r19 is None else {
        "n_phase_shift_events": r19.metrics.get("n_phase_shift_events"),
        "max_phase_shift_event_sigma": r19.metrics.get("max_phase_shift_event_sigma"),
        "secondary_significance_sigma": r19.metrics.get("secondary_significance_sigma"),
    },
    "sector_ephemeris_metrics": [m.to_dict() for m in (wf_ext.per_sector.sector_ephemeris_metrics if wf_ext.per_sector else [])],
}

## 7) Robustness checks

### 7.1 V16 sensitivity after transit-aware detrending

We apply per-sector transit-aware flattening and compare V16 results on the original vs detrended series.


In [ ]:
lc_by_sector_detrended: dict[int, btv.LightCurve] = {}

for sector, lc in sorted(ds.lc_by_sector.items()):
    t = np.asarray(lc.time, dtype=np.float64)
    f = np.asarray(lc.flux, dtype=np.float64)
    e = np.asarray(lc.flux_err, dtype=np.float64)
    in_tr = btv.get_in_transit_mask(t, ephem.period_days, ephem.t0_btjd, ephem.duration_hours)

    if btv.WOTAN_AVAILABLE:
        f_flat, trend = btv.wotan_flatten(
            t,
            f,
            window_length=0.7,
            method="biweight",
            transit_mask=in_tr,
            return_trend=True,
        )
        e_flat = e / trend
    else:
        f_flat = btv.flatten(t, f, window_length=0.7)
        e_flat = e

    lc_by_sector_detrended[int(sector)] = btv.LightCurve(time=t, flux=f_flat, flux_err=e_flat)

stitched_det = btv.stitch_lightcurves(
    [
        {
            "time": np.asarray(lc.time, dtype=np.float64),
            "flux": np.asarray(lc.flux, dtype=np.float64),
            "flux_err": np.asarray(lc.flux_err, dtype=np.float64),
            "sector": int(sector),
            "quality": np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(lc_by_sector_detrended.items())
    ]
)
lc_det = btv.LightCurve(time=stitched_det.time, flux=stitched_det.flux, flux_err=stitched_det.flux_err)

v16_orig = btv.vet_candidate(lc_stitched, candidate, preset="extended", checks=["V16"], network=False)
v16_det = btv.vet_candidate(lc_det, candidate, preset="extended", checks=["V16"], network=False)

def _v16_summary(bundle: btv.VettingBundleResult) -> dict:
    r = bundle.get_result("V16")
    return {
        "winner": None if r is None else r.metrics.get("winner"),
        "label": None if r is None else r.metrics.get("model_competition_label"),
        "winner_margin_bic": None if r is None else r.metrics.get("winner_margin_bic"),
        "artifact_risk": None if r is None else r.metrics.get("artifact_risk"),
    }

{
    "v16_original": _v16_summary(v16_orig),
    "v16_detrended": _v16_summary(v16_det),
}

### 7.2 V19 phase-shift events

V19 surfaces non-transit phase structure (harmonic scores + phase-shift event counts).


In [ ]:
v19 = btv.vet_candidate(lc_stitched, candidate, preset="extended", checks=["V19"], network=False)
r19 = v19.get_result("V19")
r19.metrics if r19 is not None else None

## 8) Optional: multi-sector pixel vetting (V08–V10)

The bundled dataset includes only a sector 83 TPF stamp. For sector-to-sector pixel consistency, download TPFs per sector.


In [ ]:
if RUN_MULTI_SECTOR_TPFS:
    if not NETWORK:
        raise ValueError("Set NETWORK=True to download TPFs")
    import lightkurve as lk

    tpf_by_sector: dict[int, btv.TPFStamp] = {}
    for sector in sorted(ds.lc_by_sector.keys()):
        search = lk.search_targetpixelfile(f"TIC {TIC_ID}", sector=int(sector), exptime=120)
        if len(search) == 0:
            print(f"No TPF for sector {sector}")
            continue
        tpf = search.download()
        if tpf is None:
            print(f"Download failed for sector {sector}")
            continue
        tpf_by_sector[int(sector)] = btv.TPFStamp(
            time=np.asarray(tpf.time.value, dtype=np.float64),
            flux=np.asarray(tpf.flux.value, dtype=np.float64),
            flux_err=np.asarray(tpf.flux_err.value, dtype=np.float64),
            wcs=tpf.wcs,
            aperture_mask=np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=np.asarray(tpf.quality, dtype=np.int32),
        )

    per = btv.per_sector_vet(
        ds.lc_by_sector,
        candidate,
        stellar=stellar,
        tpf_by_sector=tpf_by_sector,
        preset="extended",
        network=False,
    )
    for sector, bundle in sorted(per.bundles_by_sector.items()):
        r08 = bundle.get_result("V08")
        r09 = bundle.get_result("V09")
        r10 = bundle.get_result("V10")
        print(
            f"sector {sector}: V08={None if r08 is None else r08.status} "
            f"V09={None if r09 is None else r09.status} "
            f"V10={None if r10 is None else r10.status}"
        )
else:
    print("Skipping multi-sector TPF download (set RUN_MULTI_SECTOR_TPFS=True).")

## 9) Optional: AO-assisted FPP (TRICERATOPS+)

This uses the PHARO contrast curve and the tutorial light curves.

<details>
<summary><b>Expected Output</b> (click to expand)</summary>

TRICERATOPS is Monte Carlo–based, so exact values can vary (especially in `preset="fast"`).

A representative run with the PHARO AO contrast curve for TOI-5807.01 is:

```
FPP  ≈ 0.0044
NFPP ≈ 0.0002
P(planet) ≈ 0.976
Disposition: VALIDATED
```
</details>


In [ ]:
if RUN_FPP:
    if not NETWORK:
        raise ValueError("Set NETWORK=True for FPP")

    cc = btv.load_contrast_curve_exofop_tbl(ds.root / "PHARO_Kcont_plot.tbl", filter="Kcont")
    cache_dir = Path(tempfile.mkdtemp(prefix="btv_tutorial_fpp_"))
    cache = btv.hydrate_cache_from_dataset(
        dataset=ds,
        tic_id=TIC_ID,
        flux_type="pdcsap",
        cache_dir=cache_dir,
        cadence_seconds=120.0,
    )
    print(f"Cache directory: {cache_dir}")

    fpp = calculate_fpp(
        cache=cache,
        tic_id=TIC_ID,
        period=ephem.period_days,
        t0=ephem.t0_btjd,
        depth_ppm=float(candidate.depth_ppm or DEPTH_PPM),
        duration_hours=ephem.duration_hours,
        sectors=sorted(ds.lc_by_sector.keys()),
        stellar_radius=stellar.radius,
        stellar_mass=stellar.mass,
        preset=FPP_PRESET,
        timeout_seconds=FPP_TIMEOUT_SECONDS,
        replicates=FPP_REPLICATES,
        seed=FPP_SEED,
        contrast_curve=cc,
    )

    fpp
else:
    print("Skipping FPP (set RUN_FPP=True).")

## 10) Export shareable artifacts (Markdown / CSV / JSON)

This writes to a temporary directory by default.

<details>
<summary><b>Expected Output</b> (click to expand)</summary>

The export cell returns a temp directory path and writes:

- `report.md`
- `checks.csv`
- `bundle.json`

</details>


In [ ]:
out_dir = Path(tempfile.mkdtemp(prefix="btv_toi5807_"))

btv.export_bundle(wf_ext.bundle, format="md", path=out_dir / "report.md", title=f"{TOI_LABEL} ({TIC_ID})")
btv.export_bundle(wf_ext.bundle, format="csv", path=out_dir / "checks.csv")
btv.export_bundle(wf_ext.bundle, format="json", path=out_dir / "bundle.json")

str(out_dir)